In [0]:
import numpy as np
import pandas
import math
import cv2
import matplotlib.pyplot as plt
from scipy.fftpack import dct
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.svm import SVC
from google.colab import drive
from sklearn.metrics import accuracy_score
drive.mount('/content/gdrive')
from os import listdir
from os.path import isfile, join
from PIL import Image
train_path = "Devnagari/Train/" # Training folder
test_path = "Devnagari/Test/" # Testing folder

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
x_train = []
x_test = []
y_train = [] #class lables
y_test = []
for folder in range(10):
  count_f = 0
  for f in listdir(train_path+str(folder)+"/"):
    image = np.array(Image.open(train_path+str(folder)+"/"+f))
    image = cv2.resize(image, dsize=(32,32), interpolation=cv2.INTER_NEAREST)
    x_train.append(image.flatten())
    y_train.append(folder)
    count_f = count_f + 1
    if count_f == 400: # we will load only 400 images per class for training
      break;
for folder in range(10):
  count_f = 0
  for f in listdir(test_path+str(folder)+"/"):
    image = np.array(Image.open(test_path+str(folder)+"/"+f))
    image = cv2.resize(image, dsize=(32,32), interpolation=cv2.INTER_NEAREST)
    x_test.append(image.flatten())
    y_test.append(folder)
    count_f = count_f + 1
    if count_f == 100: # we will load only 100 images per class
      break;

In [0]:
# Without Kernel
clf = SVC(kernel = 'linear')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

0.898


In [0]:
# Poly Kernel
clf = SVC(kernel = 'poly')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.914


In [0]:
# RBF Kernel
clf = SVC(gamma='auto')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

0.1


In [0]:
# Cosine Kernel
norm_train = np.zeros(len(x_train))
norm_test = np.zeros(len(x_test))
for i in range(len(x_train)):
  norm_train[i] = np.linalg.norm(x_train[i])
for i in range(len(x_test)):
  norm_test[i] = np.linalg.norm(x_test[i])

try:
  del gram_mat
except:
  print("y")
gram_mat = np.zeros([len(x_train),len(x_train)]) # Gram Matrix for Testing
for i in range(gram_mat.shape[0]):
  for j in range(i,gram_mat.shape[1]):
    temp = np.dot(x_train[i],x_train[j])/(norm_train[i]*norm_train[j]) # Cosine Function
    gram_mat[i,j] = temp
    gram_mat[j,i] = temp
# Training
clf = SVC(kernel = "precomputed")
clf.fit(gram_mat,y_train)

try:
  del pred_mat
except:
  print("y")
pred_mat = np.zeros([len(x_test),len(x_train)]) # Prediction Matrix for Testing
for i in range(pred_mat.shape[0]):
  for j in range(pred_mat.shape[1]):
    temp = np.dot(x_test[i],x_train[j])/(norm_test[i]*norm_train[j]) # Cosine Function
    pred_mat[i,j] = temp
# Testing
y_pred = clf.predict(pred_mat)
print(accuracy_score(y_test, y_pred))

0.09


In [0]:
# Tanh Kernel
a = 0.005
r = -3

try:
  del gram_mat
except:
  print("y")
gram_mat = np.zeros([len(x_train),len(x_train)]) # Gram Matrix for Testing
for i in range(gram_mat.shape[0]):
  for j in range(i,gram_mat.shape[1]):
    temp = np.tanh((a*np.dot(x_train[i],x_train[j])) + r) # Tanh Function
    gram_mat[i,j] = temp
    gram_mat[j,i] = temp
# Training
clf = SVC(kernel = "precomputed")
clf.fit(gram_mat,y_train)

try:
  del pred_mat
except:
  print("y")
pred_mat = np.zeros([len(x_test),len(x_train)]) # Prediction Matrix for Testing
for i in range(pred_mat.shape[0]):
  for j in range(pred_mat.shape[1]):
    temp = np.tanh((a*np.dot(x_test[i],x_train[j])) + r) # Tanh Function
    pred_mat[i,j] = temp
# Testing
y_pred = clf.predict(pred_mat)
print(accuracy_score(y_test, y_pred))

0.096
